# Getting Started

<a name='contents'></a>
## Contents

1. <a href='#magic'>The <tt>%matmodlab</tt> Magic</a>
1. <a href='#model.def'>Defining a Model</a>
   1. <a href='#model.def.mat'>Material Model Definition</a>
   2. <a href='#model.def.step'>Step Definitions</a>
2. <a href='#model.run'>Running a Model</a>
3. <a href='#model.out'>Model Outputs</a>
4. <a href='#model.view'>Viewing Model Results</a>

<a name='magic'></a>
## The %matmodlab Magic

A notebook should include the IPython magic

    %matmodlab

near its beginning to populate its namespace with Matmodlab specific parameters, classes, and symbolic constants.

In [1]:
%matmodlab

Loading BokehJS ...

Populating the interactive namespace from matmodlab and bokeh


If the `%matmodlab` magic function cannot be found by IPython, check to be sure that the notebook server was launched by `mml notebook`.

<a name='model.def'></a>
## Defining a Model

The purpose of a Matmodlab model is to predict the response of a material to deformation.  A Matmodlab model requires two parts to be fully defined: 

- *Material* model: the material type and associated parameters.
- Deformation *step[s]*: defines deformation paths through which the material model is exercised.

The `MaterialPointSimulator` object manages and allocates memory for materials and analysis steps. Minimally, instantiating a `MaterialPointSimulator` object requires a simulation ID:

In [2]:
mps = MaterialPointSimulator('jobid')

Other optional arguments to `MaterialPointSimulator` are

- `output_format` defines the output format of the simulation results.  Valid choices are `REC` [default] and `TXT`.
- `d` specifies the directory to which simulation results are written.  The default is the current directory.

**Note:** by default results *are not* written when exercised from the Notebook.  If written results are required, the `MaterialPointSimulator.dump` method must be called explicitly.

<a name='model.def.mat'></a>
### Material model definition

The material model is defined in Matmodlab by the `MaterialPointSimulator.Material` factory method.  The `Material` fully defines the mechanical response of the material and requires a material type and parameters:

In [3]:
parameters = {'K': 1.3e11, 'G': 5.3e10}
mps.Material('pyelastic', parameters)

The `pyelastic` type is a linear elastic model implemented in Python.  The source code is contained in `matmodlab/materials/mat_pyelastic.py`.  The parameters `K` and `G` represent the bulk and shear modulus of the material, respectively.

<a name='model.def.step'></a>
### Step Definitions

Deformation steps define the components deformation and/or stress to be seen by the material model.  Deformation steps are defined by the `MaterialPointSimulator.AnalysisStep` factory method and several convenience functions:

- `MaterialPointSimulator.StrainStep`: defines the components of strain
- `MaterialPointSimulator.StrainRateStep`: defines the components of strain rate
- `MaterialPointSimulator.StressStep`: defines the components of stress
- `MaterialPointSimulator.StressRateStep`: defines the components of stress rate
- `MaterialPointSimulator.MixedStep`: defines the mixed components of stress and strain (and their rates)

Minimally, each of the preceding `Step` methods requires the six independent components of the associated second order tensor, arranged as the `XX`, `YY`, `ZZ`, `XY`, `YZ`, `XZ` components.  For example, a step of uniaxial strain is defined by:

In [4]:
ea = .1
mps.StrainStep(components=(ea, 0, 0, 0, 0, 0))

There is not limit to the number of steps in a model and steps are run in the order defined.  To reverse the step of uniaxial strain defined in the previous cell to a state of zero strain, simply define another step in which all components of strain are zero: 

In [5]:
mps.StrainStep(components=(0, 0, 0))

If `3 =< len(components) < 6`, the missing components are assumed to be zero (if `len(components)=1`, it is assumed to be volumetric strain).  

From elementary linear elasticity, the axial and lateral stresses associated with the step of uniaxial strain are

In [6]:
sa = (parameters['K'] + 4 * parameters['G'] / 3) * ea
sl = (parameters['K'] - 2 * parameters['G'] / 3) * ea

Using a `StressStep`, an equivalent deformation path can be defined as

In [7]:
mps.StressStep(components=(sa, sl, sl), frames=50)
mps.StressStep(components=(0, 0, 0), frames=50)

The optional `frames` keyword was passed to the `StressStep` which instructs the `MaterialPointSimulator` object to perform the step in `frames` increments (50 in this case).  For `Stress` and `Mixed` steps, it is a good idea to increase the number of `frames` since the solution procedure involves a nonlinear Newton solve.

The `MixedStep` allows defining mixed-mode deformations of stress and strain.  The previous deformation path could have been defined by

In [8]:
mps.MixedStep(components=(ea, sl, sl), frames=50, descriptors="ESS")
mps.MixedStep(components=(0, 0, 0), frames=50, descriptors="ESS")

The keyword `descriptors` instructs the `MaterialPointSimulator` the intent of each `component`.  The i$^{\rm th}$ descriptor corresponds to the i$^{\rm th}$ component with `E` representing strain and `S` representing stress.

The deformation path can be defined equivalently through the specification of stress and strain rate steps:

In [9]:
mps.StrainRateStep(components=(ea, 0, 0), frames=50)
mps.StrainRateStep(components=(ea, 0, 0), frames=50, scale=-1)
mps.StressRateStep(components=(sa, sl, sl), frames=50)
mps.StressRateStep(components=(sa, sl, sl), frames=50, scale=-1)

The keyword `scale` is a scale factor applied to each of the components of `components`.

Two other factory methods allow defining components of the deformation gradient and displacement:

- `MaterialPointSimulator.DefGradStep`: defines the components of the deformation gradient
- `MaterialPointSimulator.DisplacementStep`: defines the components of displacement

The `DefGradStep` method requires the nine components of the deformation gradient, arranged in row-major fashion.  The `DisplacementStep` method requires the three components of the displacement.

In [10]:
fa = exp(ea)
mps.DefGradStep(increment=1., components=((fa,0,0),(0,1,0),(0,0,1)))
mps.DefGradStep(increment=1., components=((1,0,0),(0,1,0),(0,0,1)))

<a name='model.run'></a>
## Running the Model

Deformation steps are run when they are created.

<a name='model.out'></a>
## Model Outputs

Model outputs computed by the `MaterialPointSimulator` are stored in the `records` object:

In [11]:
mps.records.keys()

['Step', 'Frame', 'Time', 'DTime', 'S', 'E', 'F', 'D', 'DS', 'EF', 'T']

Model outputs are stored as attributes to the `MaterialPointSimulator`.  For example, the components of stress throughout the history of the simulation is:

In [12]:
s = mps.S

Individual components are also accessed by attribute:

In [13]:
sxx = mps.S.XX
assert (amax(sxx) - sa) / amax(sxx) < 1e-8

Equivalently, the `MaterialPointSimulator.get` method can retrieve components field outputs from the output database:

In [14]:
sxx_ = mps.get('S.XX')
assert allclose(sxx_, sxx)

<a name='model.view'></a>
## Viewing Model Outputs

Matmodlab leverages the `bokeh` and `matplotlib` plotting libraries for displaying model outputs.  By default, the `bokeh` library is loaded by the `%matmodlab` magic and `matplotlib` is loaded if `bokeh` cannot be found.  Optionally, `matplotlib` can be explicitly requested as the default plotting library by specifying 

```python
    %matmodlab matplotlib
```

The simplest method of viewing model outputs is using the `MaterialPointSimulator.plot` method:

In [15]:
p = mps.plot('Time', 'E.XX')
show(p);

`MaterialPointSimulator.plot` takes as string arguments the `x` and `y` `MaterialPointSimulator` attributes to plot.  Incidentally (and as expected) the plot shows that axial compoent of strain was cycled from 0-10% strain through a variety of deformation steps. 

Greater flexibility in plotting can be obtained by generating a `plot` object and directly plotting variables of interest

In [16]:
p = create_figure(x_axis_label='Time', y_axis_label='Stress')
p.line(mps.Time, mps.S.XX, color='blue')
p.line(mps.Time, mps.S.YY, color='red')
show(p)

## Where to Next?

[User Defined Materials](UserMaterials.ipynb)